<a href="https://colab.research.google.com/github/denskoy/dolzhenkov_task/blob/main/Temp_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install pyspark

In [15]:
from os import sep
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .appName("films_actors") \
        .getOrCreate()
df_movies= spark.read.csv("/content/movies.csv", header = True, sep = ",")
df_actors= spark.read.csv("/content/actors.csv", header = True, sep = ",")
df_movie_actors= spark.read.csv("/content/movie_actors.csv", header = True, sep = ",")
df_movies.createOrReplaceTempView("movies")
df_actors.createOrReplaceTempView("actors")
df_movie_actors.createOrReplaceTempView("movie_actors")
# Топ-5 жанров по количеству фильмов
df_top_genre = spark.sql("""
SELECT genre, COUNT(movie_id) AS num_movies
FROM movies
GROUP BY genre
ORDER BY COUNT(movie_id) DESC
""")
df_top_genre.show()
# Актер с наибольшим количеством фильмов
top_actor = spark.sql("""
SELECT a.name, COUNT(*) as num_movies
from actors a
JOIN movie_actors ma on a.actor_id = ma.actor_id
GROUP BY a.name
ORDER BY num_movies DESC
limit 1
""")
top_actor.show()
# Средний бюджет фильмов по жанрам
avg_budget = spark.sql("""
SELECT genre, avg(budget)
FROM movies
GROUP BY genre
""")
avg_budget.show()
# Фильмы с более чем одним актером из одной страны
movies_with_multiple_actors_same_country = spark.sql("""
SELECT m.title, a.country, COUNT(*) as num_actors
FROM movies m
JOIN movie_actors ma ON m.movie_id = ma.movie_id
JOIN actors a ON ma.actor_id = a.actor_id
GROUP BY m.title, a.country
HAVING num_actors > 1
""")

movies_with_multiple_actors_same_country.show()

+------+----------+
| genre|num_movies|
+------+----------+
| Drama|         6|
|Action|         6|
|Comedy|         4|
|Horror|         2|
|Sci-Fi|         2|
+------+----------+

+--------+----------+
|    name|num_movies|
+--------+----------+
|Actor_17|         5|
+--------+----------+

+------+--------------------+
| genre|         avg(budget)|
+------+--------------------+
| Drama| 6.076021856166667E7|
|Horror|      8.7281876775E7|
|Comedy|     5.20709662225E7|
|Action|2.7492742561666667E7|
|Sci-Fi|       7.809715175E7|
+------+--------------------+

+--------+---------+----------+
|   title|  country|num_actors|
+--------+---------+----------+
| Movie_7|    India|         2|
| Movie_3|      USA|         2|
|Movie_10|       UK|         2|
|Movie_15|    India|         2|
|Movie_18|Australia|         2|
| Movie_1|    India|         3|
| Movie_2|      USA|         2|
| Movie_7|      USA|         2|
|Movie_10|      USA|         2|
+--------+---------+----------+

